Team: Eagles

Important Missing Data: Interceptions, Incomplete Passes, QB Fumbles (non-scramble)

1a. Eagles 3rd Down Conversion % by Week (Weeks 1–9), Ranked:

   Rank  Week    Attempts     Conversions      3rd Down Conversion Rate
     1     1        13           10                    76.92
     2     2        12            8                    66.67
     3     5         8            5                    62.50
     4     6        10            6                    60.00
     5     8        10            6                    60.00
     6     3         9            5                    55.56
     7     9         9            5                    55.56
     8     4        10            4                    40.00

1b.  Week Opponent  Eagles Score  Opp.Score   Result
     1      DET         38         35         Win
     2      MIN         24          7         Win
     3      WAS         24          8         Win
     4      JAX         29         21         Win
     5      ARI         20         17         Win
     6      DAL         26         17         Win
     8      PIT         35         13         Win
     9      HOU         29         17         Win

1c. Eagles Turnover Differential per Game (non-QB (except scrambles) FUMBLES lost only):

Week Opponent  Takeaways  Giveaways  Turnover Differential
1      DET          0          0              0
2      MIN          0          0              0
3      WAS          0          0              0
4      JAX          0          0              0
5      ARI          0          0              0
6      DAL          0          0              0
8      PIT          0          0              0
9      HOU          0          0              0

2. What were the most important factors for success for the team you chose?

– Most importantly, by far, were fumbles lost. The top 26 plays in -EPA were fumbles lost. The Eagles didn't lose a single fumble in the dataset. Of course, you removed interceptions which would also rank very highly. 
– 3rd Down Conversion Rate. The correlation between team winning % and 3rd down conversion rate was .375. There may be some bias in better teams converting at a higher rate, but I still think this is high enough to be noteworthy. The Eagles ranked 2nd in 3rd down conversions league-wide. 
– Explosive plays. 25 of the top 27 plays in EPA were long TDs. The Bills, Ravens, and Bengals all had mulitple of those 27 (not the Eagles, but 3 of the top offenses). 


In [81]:
import pandas as pd

#WIN/LOSS

games = pd.read_csv("~/Downloads/Technical Assessment Data 2025/games.csv")
plays = pd.read_csv("~/Downloads/Technical Assessment Data 2025/plays.csv")

# weeks 1-9
eagles_games = games[
    ((games['homeTeamAbbr'] == 'PHI') | (games['visitorTeamAbbr'] == 'PHI')) &
    (games['week'] <= 9)
].copy()

plays = plays[plays['gameId'].isin(eagles_games['gameId'])].copy()

# convert numeric
plays['preSnapHomeScore'] = pd.to_numeric(plays['preSnapHomeScore'], errors='coerce')
plays['preSnapVisitorScore'] = pd.to_numeric(plays['preSnapVisitorScore'], errors='coerce')

# max score for each team
final_scores = plays.groupby('gameId').agg(
    finalHomeScore=('preSnapHomeScore', 'max'),
    finalVisitorScore=('preSnapVisitorScore', 'max')
).reset_index()

eagles_games = eagles_games.merge(final_scores, on='gameId', how='left')

# append scores
eagles_games['PHI_score'] = eagles_games.apply(
    lambda row: row['finalHomeScore'] if row['homeTeamAbbr'] == 'PHI' else row['finalVisitorScore'],
    axis=1
)
eagles_games['opp_score'] = eagles_games.apply(
    lambda row: row['finalVisitorScore'] if row['homeTeamAbbr'] == 'PHI' else row['finalHomeScore'],
    axis=1
)

# win/loss
eagles_games['opponent'] = eagles_games.apply(
    lambda row: row['visitorTeamAbbr'] if row['homeTeamAbbr'] == 'PHI' else row['homeTeamAbbr'],
    axis=1
)
eagles_games['result'] = eagles_games.apply(
    lambda row: 'Win' if row['PHI_score'] > row['opp_score'] else 'Loss',
    axis=1
)

result = eagles_games[['week', 'opponent', 'PHI_score', 'opp_score', 'result']].sort_values('week')
print(result)


   week opponent  PHI_score  opp_score result
0     1      DET         38         35    Win
1     2      MIN         24          7    Win
2     3      WAS         24          8    Win
3     4      JAX         29         21    Win
4     5      ARI         20         17    Win
5     6      DAL         26         17    Win
6     8      PIT         35         13    Win
7     9      HOU         29         17    Win


In [82]:
# eagles 1-9
eagles_games = games[
    ((games['homeTeamAbbr'] == 'PHI') | (games['visitorTeamAbbr'] == 'PHI')) &
    (games['week'] <= 9)
][['gameId', 'week']].copy()

plays = plays[plays['gameId'].isin(eagles_games['gameId'])].copy()
plays = plays.merge(eagles_games, on='gameId', how='left')

# next down
plays.sort_values(['gameId', 'playId'], inplace=True)
plays['next_down'] = plays.groupby('gameId')['down'].shift(-1)

# 3rd down, not nullified
third_down = plays[
    (plays['possessionTeam'] == 'PHI') &
    (plays['down'] == 3) &
    (plays['playNullifiedByPenalty'] == 'N')
].copy()

third_down['playResult'] = pd.to_numeric(third_down['playResult'], errors='coerce')
third_down['yardsToGo'] = pd.to_numeric(third_down['yardsToGo'], errors='coerce')

# conversion function
def is_conversion(row):
    if row['playResult'] > row['yardsToGo']:
        return True
    elif row['playResult'] < row['yardsToGo']:
        return False
    else:
        if pd.isnull(row['next_down']):
            return False
        elif row['next_down'] in [1, 4]:
            return True
        else:
            return False

third_down['converted'] = third_down.apply(is_conversion, axis=1)

conversion_stats = (
    third_down
    .groupby('week', as_index=False)
    .agg(
        attempts=('playId', 'count'),
        conversions=('converted', 'sum')
    )
)
conversion_stats['3rd_down_conversion_pct'] = (
    conversion_stats['conversions'] / conversion_stats['attempts'] * 100
).round(2)

conversion_stats = conversion_stats.sort_values(by='3rd_down_conversion_pct', ascending=False)
conversion_stats['rank'] = range(1, len(conversion_stats) + 1)

print("Eagles 3rd Down Conversion % by Week (Weeks 1–9), Ranked:")
print(conversion_stats[['rank', 'week', 'attempts', 'conversions', '3rd_down_conversion_pct']])

print("\nFull details of Eagles 3rd Down Attempts:")
print(third_down[['gameId', 'playId', 'week', 'playDescription', 'playResult', 'yardsToGo', 'next_down', 'converted']].to_string(index=False))


Eagles 3rd Down Conversion % by Week (Weeks 1–9), Ranked:
   rank  week  attempts  conversions  3rd_down_conversion_pct
0     1     1        13           10                    76.92
1     2     2        12            8                    66.67
4     3     5         8            5                    62.50
5     4     6        10            6                    60.00
6     5     8        10            6                    60.00
2     6     3         9            5                    55.56
7     7     9         9            5                    55.56
3     8     4        10            4                    40.00

Full details of Eagles 3rd Down Attempts:
    gameId  playId  week                                                                                                                                                                                                                      playDescription  playResult  yardsToGo  next_down  converted
2022091104     462     1                  

In [83]:
games = pd.read_csv("~/Downloads/Technical Assessment Data 2025/games.csv")
plays = pd.read_csv("~/Downloads/Technical Assessment Data 2025/plays.csv")

plays['playDescription'] = plays['playDescription'].fillna("")

# find lost fumbles
def lost_fumble(desc: str) -> bool:
    return ("FUMBLES" in desc) and ("RECOVERED by" in desc)

turnover_events = plays[plays['playDescription'].apply(lost_fumble)].copy()


giveaways = (
    turnover_events
    .groupby(['gameId', 'possessionTeam'])
    .size()
    .reset_index(name='giveaways')
)

takeaways = (
    turnover_events
    .groupby(['gameId', 'defensiveTeam'])
    .size()
    .reset_index(name='takeaways')
)

giveaways = giveaways.rename(columns={'possessionTeam': 'team'})
takeaways = takeaways.rename(columns={'defensiveTeam': 'team'})

turnover_diff = pd.merge(giveaways, takeaways, on=['gameId', 'team'], how='outer')
turnover_diff['giveaways'] = turnover_diff['giveaways'].fillna(0).astype(int)
turnover_diff['takeaways'] = turnover_diff['takeaways'].fillna(0).astype(int)
turnover_diff['turnover_diff'] = turnover_diff['takeaways'] - turnover_diff['giveaways']


turnover_diff = turnover_diff.merge(games[['gameId', 'week', 'homeTeamAbbr', 'visitorTeamAbbr']],
                                    on='gameId', how='left')

# find opponents
def get_opponent(row):
    if row['team'] == row['homeTeamAbbr']:
        return row['visitorTeamAbbr']
    elif row['team'] == row['visitorTeamAbbr']:
        return row['homeTeamAbbr']
    else:
        return None

turnover_diff['opponent'] = turnover_diff.apply(get_opponent, axis=1)

print("Turnover Differential per game for all teams (FUMBLES lost only):")
print(turnover_diff[['gameId', 'week', 'team', 'opponent', 'takeaways', 'giveaways', 'turnover_diff']].sort_values(['gameId', 'team']))

print("\nFull turnover play events (FUMBLES ... RECOVERED by):")
print(turnover_events[['gameId', 'playId', 'possessionTeam', 'defensiveTeam', 'playDescription']].to_string(index=False))


Turnover Differential per game for all teams (FUMBLES lost only):
        gameId  week team opponent  takeaways  giveaways  turnover_diff
0   2022090800     1  BUF       LA          0          2             -2
1   2022090800     1   LA      BUF          2          0              2
2   2022091100     1  ATL       NO          1          2             -1
3   2022091100     1   NO      ATL          2          1              1
4   2022091102     1  CHI       SF          1          0              1
..         ...   ...  ...      ...        ...        ...            ...
87  2022110600     9  LAC      ATL          1          1              0
88  2022110605     9  IND       NE          1          0              1
89  2022110605     9   NE      IND          0          1             -1
90  2022110608     9  ARI      SEA          0          1             -1
91  2022110608     9  SEA      ARI          1          0              1

[92 rows x 7 columns]

Full turnover play events (FUMBLES ... RECOVER

In [84]:
#just for eagles
eagles_games = games[
    (((games['homeTeamAbbr'] == 'PHI') | (games['visitorTeamAbbr'] == 'PHI')) &
     (games['week'] <= 9))
][['gameId', 'week', 'homeTeamAbbr', 'visitorTeamAbbr']].copy()

plays_eagles = plays[plays['gameId'].isin(eagles_games['gameId'])].copy()

plays_eagles['playDescription'] = plays_eagles['playDescription'].fillna("")

def lost_fumble(desc: str) -> bool:
    return ("FUMBLES" in desc) and ("RECOVERED by" in desc)

turnover_events = plays_eagles[plays_eagles['playDescription'].apply(lost_fumble)].copy()


phi_giveaways = (
    turnover_events[turnover_events['possessionTeam'] == 'PHI']
    .groupby('gameId').size()
    .reset_index(name='giveaways')
)


phi_takeaways = (
    turnover_events[turnover_events['defensiveTeam'] == 'PHI']
    .groupby('gameId').size()
    .reset_index(name='takeaways')
)


turnovers = eagles_games.merge(phi_giveaways, on='gameId', how='left') \
                        .merge(phi_takeaways, on='gameId', how='left')
turnovers['giveaways'] = turnovers['giveaways'].fillna(0).astype(int)
turnovers['takeaways'] = turnovers['takeaways'].fillna(0).astype(int)
turnovers['turnover_diff'] = turnovers['takeaways'] - turnovers['giveaways']


def get_opponent(row):
    if row['homeTeamAbbr'] == 'PHI':
        return row['visitorTeamAbbr']
    elif row['visitorTeamAbbr'] == 'PHI':
        return row['homeTeamAbbr']
    else:
        return None

turnovers['opponent'] = turnovers.apply(get_opponent, axis=1)


print("Eagles (PHI) Turnover Differential per Game (FUMBLES lost only):")
print(turnovers[['gameId', 'week', 'opponent', 'takeaways', 'giveaways', 'turnover_diff']].sort_values(['gameId']))


print("\nFull Turnover Play Events (Lost Fumbles) in Eagles Games:")
print(turnover_events[['gameId', 'playId', 'possessionTeam', 'defensiveTeam', 'playDescription']].to_string(index=False))


Eagles (PHI) Turnover Differential per Game (FUMBLES lost only):
       gameId  week opponent  takeaways  giveaways  turnover_diff
0  2022091104     1      DET          0          0              0
1  2022091901     2      MIN          0          0              0
2  2022092508     3      WAS          0          0              0
3  2022100209     4      JAX          0          0              0
4  2022100911     5      ARI          0          0              0
5  2022101611     6      DAL          0          0              0
6  2022103008     8      PIT          0          0              0
7  2022110300     9      HOU          0          0              0

Full Turnover Play Events (Lost Fumbles) in Eagles Games:
Empty DataFrame
Columns: [gameId, playId, possessionTeam, defensiveTeam, playDescription]
Index: []


In [87]:
games_1_9 = games[games['week'] <= 9].copy()

plays = plays[plays['gameId'].isin(games_1_9['gameId'])].copy()
plays = plays.merge(games_1_9[['gameId', 'week', 'homeTeamAbbr', 'visitorTeamAbbr']], on='gameId', how='left')

plays.sort_values(['gameId', 'playId'], inplace=True)
plays['next_down'] = plays.groupby('gameId')['down'].shift(-1)

third_down = plays[
    (plays['down'] == 3) &
    (plays['playNullifiedByPenalty'] == 'N')
].copy()

third_down['playResult'] = pd.to_numeric(third_down['playResult'], errors='coerce')
third_down['yardsToGo'] = pd.to_numeric(third_down['yardsToGo'], errors='coerce')

# conversion function again
def is_conversion(row):
    if pd.isna(row['playResult']) or pd.isna(row['yardsToGo']):
        return False
    if row['playResult'] > row['yardsToGo']:
        return True
    elif row['playResult'] < row['yardsToGo']:
        return False
    else:
        if pd.isna(row['next_down']):
            return False
        elif row['next_down'] in [1, 4]:
            return True
        else:
            return False

third_down['converted'] = third_down.apply(is_conversion, axis=1)

# aggregate for teams
conversion_stats_team = third_down.groupby('possessionTeam').agg(
    attempts=('playId', 'count'),
    conversions=('converted', 'sum')
).reset_index()
conversion_stats_team['3rd_down_conversion_pct'] = (
    conversion_stats_team['conversions'] / conversion_stats_team['attempts'] * 100
).round(2)

conversion_stats_team = conversion_stats_team.rename(columns={'possessionTeam': 'team'})

# win vs. loss
final_scores = plays.groupby("gameId").agg(
    finalHomeScore = ('preSnapHomeScore', 'max'),
    finalVisitorScore = ('preSnapVisitorScore', 'max')
).reset_index()

games_final = games_1_9.merge(final_scores, on='gameId', how='left')

# win/loss for each team
def game_result(row):
    if row['finalHomeScore'] > row['finalVisitorScore']:
        return pd.Series([row['homeTeamAbbr'], row['visitorTeamAbbr']])
    elif row['finalHomeScore'] < row['finalVisitorScore']:
        return pd.Series([row['visitorTeamAbbr'], row['homeTeamAbbr']])
    else:
        return pd.Series([None, None]) 

games_final[['winner', 'loser']] = games_final.apply(game_result, axis=1)

# records
record_dict = {}
for _, row in games_final.iterrows():
    if row['winner'] is not None:
        w = row['winner']
        l = row['loser']
        record_dict[w] = record_dict.get(w, {'wins': 0, 'losses': 0})
        record_dict[l] = record_dict.get(l, {'wins': 0, 'losses': 0})
        record_dict[w]['wins'] += 1
        record_dict[l]['losses'] += 1

record_df = pd.DataFrame([
    {'team': team, 'wins': rec['wins'], 'losses': rec['losses']} 
    for team, rec in record_dict.items()
])
record_df['win_pct'] = (record_df['wins'] / (record_df['wins'] + record_df['losses'])).round(3)


results = pd.merge(conversion_stats_team, record_df, on='team', how='outer')
results = results[['team', 'attempts', 'conversions', '3rd_down_conversion_pct', 'wins', 'losses', 'win_pct']]


print("Overall 3rd Down Conversion Rate and Record for Each Team (Weeks 1–9):")
print(results.sort_values(by='team').to_string(index=False))

corr = results[['3rd_down_conversion_pct', 'win_pct']].corr().iloc[0,1]
print("\nCorrelation between 3rd Down Conversion % and Win Percentage: {:.3f}".format(corr))


Overall 3rd Down Conversion Rate and Record for Each Team (Weeks 1–9):
team  attempts  conversions  3rd_down_conversion_pct  wins  losses  win_pct
 ARI        61           36                    59.02     2       6    0.250
 ATL        61           43                    70.49     3       4    0.429
 BAL        72           47                    65.28     5       3    0.625
 BUF        58           44                    75.86     5       2    0.714
 CAR        63           24                    38.10     3       5    0.375
 CHI        82           48                    58.54     2       6    0.250
 CIN        77           52                    67.53     6       1    0.857
 CLE        67           41                    61.19     2       6    0.250
 DAL        51           33                    64.71     5       2    0.714
 DEN        65           30                    46.15     3       4    0.429
 DET        60           33                    55.00     2       6    0.250
  GB        67   